In [1]:
import torch
import numpy as np
import scipy.stats
from scipy.stats import rankdata



In [2]:

def computeSumation(coefficients,m=20): # currently not used
    total = 0
    for num in range(0,m+1):
        x = np.array([num**i for i in range(len(coefficients)-1,-1,-1)])
        result = np.dot(x, coefficients)
        total+=result
    return total

<h1> Performance Notes</h1>
<h2> numpy time complexity chart : </h2>
<p1> Adding an element to the end of an np.array requires O(n) since the array will need to be rebuilt</p1>

In [13]:
# Example matrix (3x3)
matrix = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9]])

# Example array of thresholds (one threshold for each row)
thresholds = np.array([3, 5, 7])

# Count the number of values in each row that are less than their respective thresholds
count_less_than_threshold = (matrix <= thresholds[:, np.newaxis]).sum(axis=1)
count_less_than_threshold = (matrix <= thresholds[:, np.newaxis]).sum(axis=1)


print(count_less_than_threshold)

[3 2 1]


In [5]:
arr = np.array([0.2, 0.1, 0.4, 0.7, 0.3, 0.9])

new_arr = np.where(arr > 0.5, 1, 0)
new_arr

(array([3, 5]),)

In [ ]:

# Takes in a n by 1 vector 
def compute_f_Hat(y,z): 
    # U = [1,34,24].T()
    total = 0 
    # Effiency note? Check if this is faster? 
    # Instead of a for loop we can multiply the vector by the inverse of y and floor the vector and then compute the sum of the vector.
    # It might be more operations but the operations it self may be faster 
    for i in range(y.shape[0]):
        if (y[i]<=z):  # 
            total+=1
    return float(total)/y.shape[0]


# Takes in a n by 1 vector 
# where y is a matrix of J x N 
# where z is a vector of J x 1 
# returns a J x 1 vector 
def compute_f_Hat2(y,z): 
    total = 0 
    # Effiency note? Check if this is faster? 
    # Instead of a for loop we can multiply the vector by the inverse of y and floor the vector and then compute the sum of the vector.
    # It might be more operations but the operations it self may be faster 

    count_less_than_threshold = (y <= z[:, np.newaxis]).sum(axis=1)
    count_less_than_threshold = count_less_than_threshold/ y.shape[1]
    return count_less_than_threshold

# X represents a singular function in points on a grid 1 x M ( where M represents the number of points n the grid)
# Xn represents the matrix for all functions   N x M (Where N is the number of functions)
# U represents the matrix of random numbers pulled from the normal distrubution with shape J x M 
# Where J represents the unit vectors and M represents the number of points
# Note that in the example provided that there was 100 functions with 10 data points representing each function 
# This function will return the depth of function x in respect to all other functions in the data set

def depth2(arr, xn, u):
    depth=0

    arr_u_dot2 = np.dot(u,arr)
    xn_uj2 = np.matmul(xn,u.T).T
    depthVector = compute_f_Hat2(xn_uj2,arr_u_dot2)
    result = np.dot(depthVector, 1- depthVector)
    return result

def depth(arr, xn, u):
    depth=0

    arr_u_dot1 = np.dot(u,arr)
    xn_uj1 = np.matmul(xn,u.T).T
    depth_j = compute_f_Hat(xn_uj,arr_u_dot)
    for j in range(0,u.shape[0]):
        print(arr.shape)
        print(u[j].shape)
        arr_u_dot = np.dot(u[j],arr)  # results in a scalar
                                      # we can make this a vector of size 1 x 20


        xn_uj = np.matmul(xn,u[j].T) # results in a vector that is Where xn = N x M and u[j].T = M x 1 or (100x1)
                                     # change this to matrix u with shape M x 20   or (100x20)

        # Now for each row in the xn_uj we want to see how many of those rows are less than x_u_dot 
        # Naive approach

        # What if we were to compute_f_hat for all xn_uj 100x20 and (1x20).T 

        depth_j = compute_f_Hat(xn_uj,arr_u_dot)
        depth += depth_j / u.shape[0]
    return depth


# calculates the inner product between two functions 
# Where f and g are both vectors that represent the coefients
# This is the for the aproximation where you are given points and not the function


# We might need to add a flag for how rank method is caluclated 
def KW_H_Test(functionalDataSet: np.ndarray, UnitVectorMatrix: np.ndarray, 
              groups:np.ndarray): 
    # Takes in Xn and the group labels and pass it U 
    # Lets just assume for now that the groups are distr
    # Groups will be a 1xn array with each element refering to a row in a matrix representing its group 

    # Returns a Nx 1 vector representing the depth for each row of functionalDataset
    depthVector = np.apply_along_axis(depth,axis=1,arr = functionalDataSet,xn = functionalDataSet, u = UnitVectorMatrix)
    # I dont understand why the axis =1 because that should then be column wise but it works when axis = 1 so... axis =1 

    # Gets the rank giving us a Nx1 vector 
    rankVector = rankdata(depthVector)

    
    # there is a scipy version of kruskal wallis we can use 
    groupRankDict = dict()
    for i in range(0,groups.shape[0]): # Runs at O(N)
        oldValue = groupRankDict.get(groups[i])
        if oldValue != None: # the key value exsits 
            groupRankDict[groups[i]] = oldValue + rankVector[i]
        else:
            groupRankDict[groups[i]] = rankVector[i]

    unique, counts = np.unique(groups, return_counts=True) # Runs at O(G)
    
    summation = 0 
    for key, value in dict(zip(unique, counts)).items(): # Runs at O(N)
        summation += (groupRankDict[key] **2) / value
    


    
    # Why 12 ? Why 3 ? 
    hStat =  12 /(groups.size* (groups.size +1)) *summation - (3*(groups.size +1))
    degreeOfFreedom = unique.size -1 
    chiSquare = scipy.stats.chi2.ppf(1-.05, df=degreeOfFreedom)
    return(hStat>chiSquare)

    # summation = ( ((group1Total**2)/3) + ((group2Total**2)/3) + ((group3Total**2)/4))
    # hStat = 12/(10*11) * summation - 3*(10+1)
    # degreeOfFreedom = 3 -1 
    # chiSquare = scipy.stats.chi2.ppf(1-.05, df=degreeOfFreedom)

    # return(hStat>chiSquare)


Xn = np.random.random((100, 10)) # represents all the data  in points
U = np.random.random((20, 10))   # unit matrix
groups = np.random.randint(1, 11, size=100) # where size must be the amount of rows in Xn 

temp = KW_H_Test(Xn,U,groups)

(20, 100)
(10,)
(10,)


Traceback (most recent call last):
  File "/opt/anaconda3/envs/eecs4422/lib/python3.9/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_vars.py", line 478, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
  File "<string>", line 1, in <module>
NameError: name 'array' is not defined


(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(20, 100)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(20, 100)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(20, 100)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(20, 100)
(10,)
(10,)


<h1> Running the test </h1>

IndexError: invalid index to scalar variable.

In [ ]:
groups = np.random.randint(1, 11, size=100) # where size must be the amount of rows in Xn 

temp = KW_H_Test(Xn,U,groups)

In [52]:
unique, counts = np.unique(groups, return_counts=True)

dict(zip(unique, counts))

{1: 19, 2: 6, 3: 10, 4: 11, 5: 14, 6: 6, 7: 11, 8: 6, 9: 10, 10: 7}

In [55]:
unique.size

10

In [39]:
print(temp[2])

{1: 454.0, 2: 696.0, 5: 481.0, 9: 561.0, 10: 425.0, 6: 319.0, 7: 1042.0, 3: 255.0, 8: 562.0, 4: 255.0}


In [40]:
import pandas as pd
ranks, groups, sum1= temp
df = pd.DataFrame({'Column1': groups, 'Column2': ranks})

In [ ]:
sum1

In [ ]:
total = 0 

for i in sum1.values():
    total += i
print(total)

In [ ]:
my_dict = dict(zip(groups, ranks))

In [ ]:

# Unzip the tuple into two separate vectors (arrays)
vector1, vector2 = temp

In [ ]:
# print(data.shape)
# print(data["MCO"])

In [ ]:
import skfda
data = skfda.datasets.fetch_cran("MCO", "fda.usc")


In [ ]:
matrix1 = np.random.random((100, 10))

# Create a 10x20 matrix
matrix2 = np.random.random((10, 20))

# Perform matrix multiplication
result = np.matmul(matrix1, matrix2)
print(result.shape[1])

In [ ]:
import numpy as np
from scipy.stats import rankdata

# Sample NumPy array of tuples
data = np.array([(10, 'A'), (5, 'B'), (15, 'C'), (8, 'D')])

# Extract the numerical values from the tuples and rank them
numerical_values = data[:, 0]  # Extract the first element of each tuple (numerical value)
ranks = rankdata(numerical_values)

# Create an array of tuples with ranks along with the original data
ranked_data = np.column_stack((data, ranks))

print("Original Data:")
print(data)

print("\nRanked Data:")
print(ranked_data)
